In [1]:
import numpy as np

In [2]:
board = np.array([
    [ 57,  33, 132, 268, 492, 732], 
    [ 81, 123, 240, 443, 353, 508], 
    [186,  42, 195, 704, 452, 228], 
    [ -7,   2, 357, 452, 317, 395], 
    [  5,  23,  -4, 592, 445, 620], 
    [  0,  77,  32, 403, 337, 452]
])[::-1, :]

In [3]:
board[::-1, :]

array([[ 57,  33, 132, 268, 492, 732],
       [ 81, 123, 240, 443, 353, 508],
       [186,  42, 195, 704, 452, 228],
       [ -7,   2, 357, 452, 317, 395],
       [  5,  23,  -4, 592, 445, 620],
       [  0,  77,  32, 403, 337, 452]])

In [4]:
arr = np.arange(6)
initial_die = {"num": np.zeros(6), "set": np.zeros(6, dtype=bool)}
set_die = lambda die, label, value: {"num": die["num"] * (arr != label-1) + value * (arr == label-1), 
                                     "set": die["set"] | (arr == label-1)}

In [5]:
# label_topology
antipode = {1: 6, 2: 4, 3: 5, 4: 2, 5: 3, 6: 1}
# configuration = (front, up, right)
initial_cfg = (2,1,3)
# tipping transformations
tip_front = lambda cfg: (cfg[1], antipode[cfg[0]], cfg[2])
tip_right = lambda cfg: (cfg[0], antipode[cfg[2]], cfg[1])
tip_back = lambda cfg: (antipode[cfg[1]], cfg[0], cfg[2])
tip_left = lambda cfg: (cfg[0], cfg[2], antipode[cfg[1]])

# moves on the board
initial_pos = np.array([0, 0])
final_pos = np.array([5, 5])
move_front = lambda pos: pos+np.array([1, 0])
move_right = lambda pos: pos+np.array([0, 1])
move_back = lambda pos: pos+np.array([-1, 0])
move_left = lambda pos: pos+np.array([0, -1])

def directions(pos):
    funcs = []
    if pos[0] > 0:
        funcs.append((move_back, tip_back))
    if pos[1] > 0:
        funcs.append((move_left, tip_left))
    if pos[0] < board.shape[0] - 1:
        funcs.append((move_front, tip_front))
    if pos[1] < board.shape[1] - 1:
        funcs.append((move_right, tip_right))
        
    return funcs

In [6]:
history = []
def backtrack(pos, cfg, die, N, history):
    if (pos == final_pos).all():
        history.append((pos, cfg, die))
        return True
    
    for move, tip in directions(pos):
        delta_score = board[tuple(move(pos))] - board[tuple(pos)]
        face_up_value = delta_score/(N+1)
        face_up_label = tip(cfg)[1]
        
        if die["set"][face_up_label - 1] and die["num"][face_up_label - 1] != face_up_value:
            continue
        
        if backtrack(move(pos), tip(cfg), set_die(die, face_up_label, face_up_value), N+1, history):
            history.append((pos, cfg, die))
            return True
    

In [7]:
backtrack(initial_pos, initial_cfg, initial_die, 0, history)

True

In [9]:
for pos, cfg, die in history[::-1]:
    print(f"""Pos (i, j) = {tuple(pos)} -> {board[tuple(pos)]:3.0f} | Cfg (f,u,r) = {cfg} | DS: {
            [f"{val:2.0f}" if s else "??" for val, s in zip(die["num"], die["set"])]
            }""")

Pos (i, j) = (0, 0) ->   0 | Cfg (f,u,r) = (2, 1, 3) | DS: ['??', '??', '??', '??', '??', '??']
Pos (i, j) = (1, 0) ->   5 | Cfg (f,u,r) = (1, 4, 3) | DS: ['??', '??', '??', ' 5', '??', '??']
Pos (i, j) = (1, 1) ->  23 | Cfg (f,u,r) = (1, 5, 4) | DS: ['??', '??', '??', ' 5', ' 9', '??']
Pos (i, j) = (1, 2) ->  -4 | Cfg (f,u,r) = (1, 2, 5) | DS: ['??', '-9', '??', ' 5', ' 9', '??']
Pos (i, j) = (0, 2) ->  32 | Cfg (f,u,r) = (4, 1, 5) | DS: [' 9', '-9', '??', ' 5', ' 9', '??']
Pos (i, j) = (0, 1) ->  77 | Cfg (f,u,r) = (4, 5, 6) | DS: [' 9', '-9', '??', ' 5', ' 9', '??']
Pos (i, j) = (1, 1) ->  23 | Cfg (f,u,r) = (5, 2, 6) | DS: [' 9', '-9', '??', ' 5', ' 9', '??']
Pos (i, j) = (2, 1) ->   2 | Cfg (f,u,r) = (2, 3, 6) | DS: [' 9', '-9', '-3', ' 5', ' 9', '??']
Pos (i, j) = (3, 1) ->  42 | Cfg (f,u,r) = (3, 4, 6) | DS: [' 9', '-9', '-3', ' 5', ' 9', '??']
Pos (i, j) = (4, 1) -> 123 | Cfg (f,u,r) = (4, 5, 6) | DS: [' 9', '-9', '-3', ' 5', ' 9', '??']
Pos (i, j) = (5, 1) ->  33 | Cfg (f,u,r)

In [10]:
eval_board = board.copy()
for pos, cfg, die in history:
    eval_board[tuple(pos)] = 0

In [13]:
eval_board[::-1, :]

array([[ 57,   0,   0, 268, 492,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 452, 228],
       [ -7,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 445,   0],
       [  0,   0,   0,   0,   0,   0]])

In [12]:
answer = eval_board.sum()
answer

1935